Notebook to train the model.

Dataset:
    lib/Datasets.py:
        EEGDataset

Models:
    lib/models/EEG_Net_CNN.py : EEG_Net_CNN - a simple convolutional neural network [conv -> conv -> conv -> dense] with ELU activation, pooling, and batch normalization

In [2]:
# Packages
from lib.models.EEG_Net_CNN import EEG_Net_CNN
import matplotlib.pyplot as plt
from lib.utils import load_data, train, test
from lib.Datasets import EEGDataset
from lib.DataObject import DataObject
import lib.DataObjectUtils as util
import torch
import pickle
import torch.nn as nn
from lib.DataHandler import DataAcquisitionHandler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import WeightedRandomSampler
import datetime

In [4]:
# load pickle files
filepath = "C:/Users/c25th/code/P300_BCI_Speller/"
date = datetime.datetime.now().strftime("%Y-%m-%d")

train_filename = "data/dataloaders/train_loader_"+str(date)+".txt"
val_filename = "data/dataloaders/val_loader_"+str(date)+".txt"
test_filename = "data/dataloaders/test_loader_"+str(date)+".txt"

train_loader = torch.load(filepath + train_filename)
val_loader = torch.load(filepath + val_filename)
test_loader = torch.load(filepath + test_filename)

In [ ]:
import torch
import torch.nn as nn

# create p300Model
class EEG_Net_CNN(torch.nn.Module):
    """
    Pytorch implementation of EEGNet

    Expecting input of shape (batch_size, channels, readings)
    input = [32, 8, 250] = [batch_size, channels, readings]
    batch_size: number of samples in a batch
    channels: number of channels in a sample (8)
    readings: number of readings in a channel (len())
    """
    
    def __init__(self, num_channels=8, num_classes=2, input_length=250):
        super(EEG_Net_CNN, self).__init__()

        # Conv1
        self.conv1 = torch.nn.Sequential(
            nn.Conv1d(num_channels, 16, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(16),
            nn.ELU(alpha=1.0),
            nn.AvgPool1d(kernel_size=4, stride=4, padding=0)
            # nn.Dropout(p=0.25)
        )

        # Conv2
        self.conv2 = torch.nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(32),
            nn.ELU(alpha=1.0),
            nn.AvgPool1d(kernel_size=4, stride=4, padding=0)
            # nn.Dropout(p=0.25)
        )

        # Conv3
        self.conv3 = torch.nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(64),
            nn.ELU(alpha=1.0),
            nn.AvgPool1d(kernel_size=4, stride=4, padding=0)
            # nn.Dropout(p=0.25)
        )

        # Conv4
        self.conv4 = torch.nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(128),
            nn.ELU(alpha=1.0),
            nn.AvgPool1d(kernel_size=4, stride=4, padding=0)
            # nn.Dropout(p=0.25)
        )

        # Conv5
        self.conv5 = torch.nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(256),
            nn.ELU(alpha=1.0),
            nn.AvgPool1d(kernel_size=4, stride=4, padding=0)
            # nn.Dropout(p=0.25)
        )

        # Output
        self.out = torch.nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        x = self.conv4(x)
        print(x.shape)
        x = self.conv5(x)
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.out(x)

        return x

In [5]:
# initialize model
model = EEG_Net_CNN()

# loss function
loss_fn = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
# def train_and_validate(train_dataloader, val_dataloader, model, loss_fn, optimizer, num_epochs, print_every=100):
train_losses, val_losses, train_accuracies, val_accuracies = train(
    train_loader, val_loader, model, loss_fn, optimizer, 100, print_every=10)

torch.Size([32, 8, 250])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x5984 and 128x2)

In [7]:
print("Maximum Train Accuracy: ", max(train_accuracies))
test(test_loader, model, loss_fn)

Maximum Train Accuracy:  0.536441828881847
Test Error: 
 Accuracy: 46.6%, Avg loss: 0.701727 

